In [1]:
'''
Set-up:
1) Threshold policy 
2) Continuous state space
3) Finite Horizon 
4) Monte Carlo approximation
'''

'\nSet-up:\n1) Threshold policy \n2) Continuous state space\n3) Finite Horizon \n4) Monte Carlo approximation\n'

In [2]:
import numpy as np
import math
np.set_printoptions(formatter={'float': lambda x: "{0:0.6f}".format(x)})
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [3]:
'''
Parameters: norizon length, demand distribution, start state distribution
'''
np.random.seed(10)
horizon = 20
demand_min = 0
demand_max = 1
start_mean = 1
start_var = 0.5
'''
Costs
'''
c = 0.5
p = 0.6
h = 0.55

## Optimal Policy

In [4]:
sims_for_MC = 500
y_min = -1
y_max = 1

In [5]:
demand_mat = np.random.uniform(demand_min,demand_max,size=(sims_for_MC,horizon))

In [6]:
def get_cost(state,action):
    return c*action + h*max(0,state) + p*max(0,-state)

In [53]:
# '''
# horizon = length if the decision horizon. Example, horizon = 3 menas decision in 0,1,2
# curr_time = current_period, so can go from 0 to horizon-1
# '''

# def q_function(y_init,theta_star,curr_time,horizon):
#     '''
#     The +1's in length is to accomodate the cost in the state after the final decison is taken
#     '''
#     variable = np.zeros(horizon-curr_time+1)
#     cost = np.zeros(horizon-curr_time+1)
#     variable[0] = y_init
#     cost[0] = c*variable[0] 
#     action = 0
    
#     '''
#     Now computing the continuation costs, J_{h+1} by forward simulation
#     '''
#     for i in range(0,horizon-curr_time-1): 
#         print('Getting here')
#         variable[i+1] = variable[i] + action - demand_mat[0,curr_time+i]
#         action = max(0,theta_star[curr_time+i+1]-variable[i+1])
#         cost[i] = get_cost(variable[i],action) 
    
#     variable[-1] = variable[horizon-curr_time-1] + action - demand_mat[0,-1]
#     print variable

In [19]:
'''
horizon = length if the decision horizon. Example, horizon = 3 menas decision in 0,1,2
curr_time = current_period, so can go from 0 to horizon-1
Inefficient code in terms of memory but easy to understand indexing
'''

def q_function(y_init,theta_star,curr_time,horizon):
    '''
    The +1's in length is to accomodate the cost in the state after the final decison is taken
    '''
    variable = np.zeros(horizon+1)
    cost = np.zeros(horizon+1)
    variable[curr_time] = y_init
    cost[curr_time] = c*variable[curr_time] 
    action = 0
    
    '''
    Now computing the continuation costs, J_{h+1} by forward simulation
    '''
    for i in range(curr_time,horizon-1): 
        print('Getting here')
        variable[i+1] = variable[i] + action - demand_mat[0,i]
        action = max(0,theta_star[i+1]-variable[i+1])
        cost[i+1] = get_cost(variable[i+1],action) 
    
    print(action)
    variable[-1] = variable[horizon-1] + action - demand_mat[0,-1]
    print variable

In [20]:
theta_star = np.zeros(horizon)

In [21]:
'''
Just a check
'''
q_function(0.5,theta_star,0,horizon)

Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
Getting here
0.9177741225129434
[0.500000 -0.271321 -0.020752 -0.633648 -0.748804 -0.498507 -0.224797
 -0.198063 -0.760531 -0.169111 -0.088340 -0.685360 -0.953393 -0.003948
 -0.512192 -0.812621 -0.612526 -0.721755 -0.291876 -0.917774 -0.714576]


## Monte Carlo approximation for Qfunction

In [7]:
'''
horizon = length if the decision horizon. Example, horizon = 3 menas decision in 0,1,2
curr_time = current_period, so can go from 0 to horizon-1
Inefficient code in terms of memory but easy to understand indexing.
Efficient code will create vectors of size horizon - curr_time + 1 but requires more complex indexing
'''

def q_function(sims_for_MC,y_init,theta_star,curr_time,horizon):
    
    total_cost = 0
    for k in range(sims_for_MC):
    
        '''
        The +1's in length is to accomodate the cost in the state after the final decison is taken
        '''
        variable = np.zeros(horizon+1)
        cost = np.zeros(horizon+1)
        variable[curr_time] = y_init
        cost[curr_time] = c*variable[curr_time] 
        action = 0

        '''
        Now computing the continuation costs, J_{h+1} by forward simulation
        '''
        for i in range(curr_time,horizon-1): 
            variable[i+1] = variable[i] + action - demand_mat[k,i]
            action = max(0,theta_star[i+1]-variable[i+1])
            cost[i+1] = get_cost(variable[i+1],action) 

        variable[-1] = variable[horizon-1] + action - demand_mat[k,-1]
        cost[-1] = get_cost(variable[-1],0)
        total_cost += np.sum(cost)
    
    return total_cost/sims_for_MC

In [8]:
theta_star = np.zeros(horizon)

## Golden Search

In [9]:
'''
Wikipedia Implementation
'''
gr = (math.sqrt(5) + 1) / 2
def gss(a, b, curr_time, tol=1e-2):
    '''
    golden section search to find the minimum of f on [a,b]
    f: a strictly unimodal function on [a,b]
    '''
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    
    while abs(c - d) > tol:
        f_c = q_function(sims_for_MC,c,theta_star,curr_time,horizon)
        f_d = q_function(sims_for_MC,d,theta_star,curr_time,horizon)
        if f_c < f_d:
            b = d
        else:
            a = c

        # we recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        c = b - (b - a) / gr
        d = a + (b - a) / gr
        
    return (b + a) / 2

In [15]:
'''
Just a check
'''
optimal = gss(y_min,y_max,2,tol=0.0001)
print(optimal)

0.514708427504


## Golden Search with MC approximation in a loop

In [11]:
theta_star = np.zeros(horizon)
for i in range(horizon-1,-1,-1):
    theta_star[i] = gss(y_min,y_max,i,tol=0.0001)

In [12]:
print(theta_star)

[0.473816 0.531530 0.514708 0.517081 0.536835 0.491116 0.462086 0.447291
 0.539767 0.497541 0.500474 0.533342 0.536621 0.516735 0.533688 0.523292
 0.499353 0.511776 0.465019 0.078027]


## Monte Carlo approximation of total cost

In [13]:
def ell_approx(theta):
    
    total_cost = 0
    for k in range(sims_for_MC):
        '''
        Simulation part
        '''
        state = np.zeros(horizon+1)
        cost = np.zeros(horizon+1)
        action = np.zeros(horizon)

        state[0] = np.random.normal(start_mean,start_var)
        for i in range(0,horizon):
            action[i] = max(0,theta[i] - state[i])
            cost[i] = c*action[i] + h*max(0,state[i]) + p*max(0,-state[i])
            state[i+1] = state[i] + action[i] - demand_mat[k,i]

        cost[-1] = h*max(0,state[-1]) + p*max(0,-state[-1])
        total_cost += np.sum(cost)
    
    return total_cost/sims_for_MC

In [22]:
ell_star = ell_approx(theta_star)
print(ell_star)

10.58018773422548
